In [1]:
import tensorflow as tf
import datetime

In [2]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_images = tf.expand_dims(train_images,-1)
test_images = tf.expand_dims(test_images,-1)

In [4]:
train_images = tf.cast(train_images/255,tf.float32)
test_images = tf.cast(test_images/255,tf.float32)

In [5]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images,test_labels))

In [7]:
# 在使用fit时repeat（）需要带参数，自定义循环时repeat（）不需要带参数
dataset = dataset.repeat().shuffle(60000).batch(128)
test_dataset = test_dataset.repeat().batch(128)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [9]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

Tensorboard可视化

In [10]:
# 为目录名添加时间
import os
log_dir = os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [11]:
# 第一个参数：事件文件存放地址  
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir,histogram_freq=1)

In [18]:
# 创建一个文件编写器
file_writer = tf.summary.create_file_writer(log_dir + '/lr')

In [19]:
# 设置为默认的文件编写器
file_writer.set_as_default()

In [20]:
def lr_sche(epoch):
    learning_rate = 0.2
    if epoch > 5:
        learning_rate = 0.02
    if epoch > 10:
        learning_rate = 0.01
    if epoch > 20:
        learning_rate = 0.005
    #将变化写入磁盘
    tf.summary.scalar('learning_rate',data = learning_rate,step = epoch)
    return learning_rate

In [21]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [23]:
model.fit(dataset,epochs=25,
          steps_per_epoch=60000//128,
          validation_data=test_dataset,
          validation_steps=10000//128,
          callbacks=[tensorboard_callback,lr_callback]
         )

Epoch 1/25
468/468 [==============================] - 47s 101ms/step - loss: 2.3133 - accuracy: 0.1035 - val_loss: 2.3125 - val_accuracy: 0.1135 35s - loss: 2.3149 - acc - ETA: 33s - loss: 2.3141 - accuracy: - ETA: 32s - loss: 2.3140 - accuracy: 0. - ETA: 32s - loss: 2.3137 - accuracy - ETA: 31s - loss: 2.3136 - accuracy: 0 - ETA: 31s - los - ETA: 28s - lo - ETA: 25s - loss: 2.3151   - ETA: 4s - loss: 2.3137 - accu - ETA: 1s - loss: 2.3135 - ac - ETA: 0s - loss: 2.313
Epoch 2/25
468/468 [==============================] - 47s 100ms/step - loss: 2.3147 - accuracy: 0.1049 - val_loss: 2.3122 - val_accuracy: 0.0975 - lo - ETA: 22s - loss: 2.3131 - accuracy: - ETA: 22s - loss: 2.3132 - accura - ETA: 21s - loss: 2.3132 - ac - ETA
Epoch 3/25
468/468 [==============================] - 51s 109ms/step - loss: 2.3135 - accuracy: 0.1033 - val_loss: 2.3056 - val_accuracy: 0.102935 - accur - ETA: - ETA: 30s -  - ETA: 6s - loss: 2.3135 - accuracy: 0.10 - ETA: 6s - l - ETA:  - ETA: 3s - loss: 2.3134  -

In [26]:
# 在notebook中显示
%load_ext tensorboard
%matplotlib inline

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
# 启动tensorboard:参数为目录地址，此处目录为logs
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1420), started 6:08:55 ago. (Use '!kill 1420' to kill it.)

In [15]:
#在浏览器中打开tensorboard
#1.打开cmd命令行，进入代码文件所在路径
#2.运行tensorboard --logdir logs命令
#3.将显示的网址复制到浏览器打开

# 记录自定义标量值

重新调整回归模型并记录自定义学习率。
1.使用创建文件编写器：tf.summary.create_file_writer()
2.定义自定义学习率功能，传递给Keras LeariningRateScheduler回调
3.在学习率功能内，用于tf.summary.scalar()记录自定义学习率
4.将LeariningRateScheduler回调传递给Model.fit()

# 自定义训练中使用TensorBoard

In [25]:
optimizer = tf.keras.optimizersmizerstimizers.Adam()

In [28]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [29]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)

In [30]:
# 初始化计算对象
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy') 
test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy') 

In [31]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels,pred)
        #loss_step = loss(model,images,labels) # 每一步的损失值
    grads = t.gradient(loss_step,model.trainable_variables) # 损失函数与可训练参数之间的梯度
    optimizer.apply_gradients(zip(grads,model.trainable_variables)) # 优化函数应用梯度进行优化
    # 汇总计算平均loss
    train_loss(loss_step)
    # 汇总计算平均acc
    train_accuracy(labels,pred)

In [32]:
def test_step(model,images,labels):
        pred = model(images)
        loss_step = loss_func(labels,pred)
        # 汇总计算平均loss
        test_loss(loss_step)
        # 汇总计算平均acc
        test_accuracy(labels,pred)

In [33]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [36]:
train_log_dir = 'logs/gradient_tape' + current_time + '/train'
test_log_dir = 'logs/gradient_tape' + current_time + '/test'

train_writer = tf.summary.create_file_writer(train_log_dir)
test_writer = tf.summary.create_file_writer(test_log_dir)

In [37]:
def train():
    for epoch in range(10):
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        with train_writer.as_default():
            tf.summary.scalar('loss',train_loss.result(),step = epoch) # 收集标量值,记录
            tf.summary.scalar('acc',train_accuracy.result(),step = epoch)
        
        print('Epoch{}  loss is {}, accuracy is {}'.format(epoch,train_loss.result(),train_accuracy.result()))
        for (batch,(images,labels)) in enumerate(test_dataset):
            test_step(model,images,labels)
            print('*',end='')
            
        with test_writer.as_default():
            tf.summary.scalar('loss',test_loss.result(),step = epoch) # 收集标量值,记录
            tf.summary.scalar('acc',test_accuracy.result(),step = epoch)
            
        print('Epoch{}  test_loss is {}, test_accuracy is {}'.format(epoch,test_loss.result(),test_accuracy.result()))
        # 重置
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [ ]:
train()